<span style="font-size:300%; text-align: center;"><b> Python Pandas Basics </b></span>

- Pandas comes from "Panel Data".
- It is a Python library designed help with this type of data.
- It is very useful for manipulating data and performing some basic analysis.
- Pandas dataframe can used directly with other libraries for analysis.
- There are two main objects: the Series and the Dataframe.


To start, let's import pandas


In [ ]:
import pandas as pd  # we import pandas "as pd", which means we refer to pandas as pd throughout the code.

# The DataFrame

To get a sense of how Pandas works, we'll look at a dataframe. Before we do, we'll construct the data with a dictionary, which should be familiar from the previous notebook:

In [ ]:
stock_dictionary = {'AAPL': [168, 170, 172], 'MSFT':[78, 80, 82],'AMZN':[1123, 1125, 1127],'KR': [22, 24, 26]}
print(stock_dictionary)

There are several ways to create a dataframe. You can do this from a dictionary like this:

In [ ]:
stock_df = pd.DataFrame(stock_dictionary)
print(stock_df)

The dataframe has four columns and three rows. I didn't specificy what this was, but we could think of it as the prices for some assets over three periods. Within the dataframe, each column can be manipulated as an individual series. 

For example, let's look at 'AAPL'

## Each column of a dataframe is a series:

In [ ]:
aapl = stock_df['AAPL'] # we take stock_df and select column 'AAPL'; we make a series called aapl

In [ ]:
print(aapl)

Something to be aware of: setting a series equal to a column does not make a copy - they refer to the same object.


What is the first row of aapl? (A series is indexed similar to a string. To call the index, we must use the iloc function)

In [ ]:
aapl.iloc[0]

It's the same as in the dataframe. What happens if we change it to zero?

In [ ]:
aapl.iloc[0] = 0

print(stock_df)

It also changes in stock df; the series aapl refers to the series stock_df['AAPL']



## Let's get some more data. An example from Alpha Vantage:

Pandas can read many different file formats. Alpha Vantage has free stock price data through an API. Here, I'll use the demo (which has a demo authorization key) to get MSFT in CSV format.

In [ ]:
df = pd.read_csv(r'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=demo&datatype=csv')

In [ ]:
print(df.head()) # df.head() gives us the top few rows. We see that the first observations are the last chronologically
# df.head(200) would give us 200 rows

In [ ]:
print(df.tail()) # we can also look at the tail

### Time Series Data : the DateTime Variable

What kind of variable is timestamp?

In [ ]:
print(df.timestamp.dtype)

Object: That's a string. We can't do any datetime operations with strings. We'll need to convert that:

In [ ]:
df['timestamp'] = pd.to_datetime(df.timestamp)
print(df.timestamp.dtype)

In [ ]:
print(df.timestamp.head()) #it looks the same but it's very different.

With timestamp as a datetime64, we can apply all kinds of time series functions:

In [ ]:
df['year'] = df.timestamp.dt.year
df['month'] = df.timestamp.dt.month
print(df.head())

I prefer the chronology to go from top to bottom, so let's sort on date:

In [ ]:
df.sort_values('timestamp').head()

If we call df again, the sort will not have been preserved:

In [ ]:
df.head()

To properly sort, we have to add "inplace=True" or set the df = df.sort_values('timestamp)

In [ ]:
df = df.sort_values('timestamp')
#df.sort_values('timestamp', inplace=True) # does the same thing

In [ ]:
df.head(10)

# Data Manipulation: Calculate Returns and Amihud (2002) Illiquidity Ratio

- Let's get returns close-to-close
- Amihud Illiquidity comes from: 

Amihud, Yakov. "Illiquidity and stock returns: cross-section and time-series effects." Journal of financial markets 5, no. 1 (2002): 31-56.

In [ ]:
df['l_close'] = df.close.shift(1)

In [ ]:
df.head(10)

In [ ]:
df['return'] = (df.close - df.l_close)/df.l_close # manipulating series is similar to manipulating variables in Stata

In [ ]:
df.head(10)

Amihud is equal to the average daily absolute return divided by total dollar volume:


$ Amihud = \dfrac{1}{T} \sum_{t=1}^T \dfrac{{\mid R_t\mid}}{Price_t \times Volume_t}$

It looks like volume in our data is already in dollars. Let's calculate Amihud monthly with a grouby()

We'll need to import numpy

In [ ]:
import numpy as np

df['absret'] = np.abs(df['return']) # I have to write it with the index because return is a reserved word for a method
df['absret_vol'] = df.absret / df.volume

In [ ]:
df.head()

In [ ]:
amihud = df.groupby(['month'])['absret_vol'].mean() # this creates a new series called amihud. it's outside of the df

In [ ]:
amihud

If we only have on observation per month, chances are we don't need an entire column in our dataframe with redundant information. This is one feature I like about Pandas over Stata. We can have multiple "datasets" open simultaneously in memory. If we want to make amihud another column in our original df, we have to apply a function:

In [ ]:
df['amihud'] = df.groupby(['month'])['absret_vol'].transform(lambda x: x.mean())

In [ ]:
df.head(20)

## Some Plotting Functions

Pandas can make graphs by using Matplotlib in the background.

What does the time series of returns look like?

In [ ]:
print(df['return'].plot.line())

Can we get a histogram?

In [ ]:
print(df['return'].plot.hist(bins=20))

## Summary Statistics

In [ ]:
df.describe() # on the whole dataframe

In [ ]:
df.close.describe() # on a series

The describe() method gives you another dataframe, which you can save

In [ ]:
summary_stats = df.describe()

This exports nicely to a variety of formats.


## Exporting Data

In [ ]:
df.to_stata('msft_returns.dta', write_index=False) #I don't want to include the index
summary_stats.to_excel('msft_sum_stats.xlsx', index=False) # note idiosyncracy between write_index in to_stata and index here

## Importing Data

In [ ]:
df = "rewritten string - we'll need to load the dataframe"
print(df)

In [ ]:
df = pd.read_stata('msft_returns.dta')

In [ ]:
df.head()